# Importing libraries

In [2]:

import pandas as pd
import re
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
import string
import matplotlib.pyplot as plt
import nltk


# importing dataset as fake and real

In [4]:
data_fake = pd.read_csv('fake.csv')
data_real = pd.read_csv('real.csv')

In [5]:
data_real.head()

,Title,Description
0,फिल्म ‘भगवत् गीता’मा धिरज अनुबन्धित,फिल्म ‘भगवत् गीता’मा नायक धिरज मगर अनुबन्धित ह...
1,पब्जी विश्वकपमा नेपाल दोस्रो,पब्जी विश्वकपमा नेपाल दोस्रो भएको छ । इन्डोनेस...
2,मोदीको मातृभक्ति : आमाको खुट्टा धोएर लिए आशीर्वाद,भारतका प्रधानमन्त्री नरेन्द्र मोदीले आफ्नी आमा...
3,‘लोकतन्त्रप्रति प्रतिबद्ध नेतृत्व आउँछ’– निधि,कांग्रेसको १४ औं महाधिवेशमा नेतृत्व चयनका लागि...
4,'पथेर' र 'कदम'लाई पहिचान पुरस्कार,यस वर्ष पहिचान पुरस्कार श्याम शाहको कथासंग्रह ...


In [6]:
data_fake.head()

,Title,Description
0,ट्रम्पडम: ट्रम्पको विदेश नीतिको जिज्ञासु संसा...,निरज श्रीवास्तव 21st Century WireIt डोनाल्ड ट्...
1,BREAKING: वफादार ट्रम्प समर्थकले महान्यायाधिवक...,"सेन जेफरसन ब्यूरेगार्ड सेसन्स III, डोनाल्ड ट्र..."
2,चीन ड्रोन ड्रामा ट्रम्पको पहिलो ट्वीट भन्दा पह...,डोनाल्ड ट्रम्पले अमेरिकी अन्डरवाटर ड्रोन जफत ग...
3,एक रक्षात्मक जोन केरीले सोध्छ कि इजरायलले आफ्न...,जोन केरीले ओबामा प्रशासन इजरायलको मित्र भएको द...
4,Reince Priebus ट्रम्पको दाबीलाई समर्थन गर्दछ क...,डोनाल्ड ट्रम्पले स्वतन्त्र प्रेस बिरूद्ध युद्ध...


# clasify gareu data fake lai 0 le real lai 1 lw

In [8]:
data_fake["class"]= 0
data_real["class"]= 1

# 2 tai data lai concatinate gareu

In [10]:
data = pd.concat([data_real,data_fake],axis=0)
data.head()

,Title,Description,class
0,फिल्म ‘भगवत् गीता’मा धिरज अनुबन्धित,फिल्म ‘भगवत् गीता’मा नायक धिरज मगर अनुबन्धित ह...,1
1,पब्जी विश्वकपमा नेपाल दोस्रो,पब्जी विश्वकपमा नेपाल दोस्रो भएको छ । इन्डोनेस...,1
2,मोदीको मातृभक्ति : आमाको खुट्टा धोएर लिए आशीर्वाद,भारतका प्रधानमन्त्री नरेन्द्र मोदीले आफ्नी आमा...,1
3,‘लोकतन्त्रप्रति प्रतिबद्ध नेतृत्व आउँछ’– निधि,कांग्रेसको १४ औं महाधिवेशमा नेतृत्व चयनका लागि...,1
4,'पथेर' र 'कदम'लाई पहिचान पुरस्कार,यस वर्ष पहिचान पुरस्कार श्याम शाहको कथासंग्रह ...,1


# droping the column named description

In [12]:
cleaned_data= data.drop(['Description'],axis=1, inplace=False)
cleaned_data.head(10)

,Title,class
0,फिल्म ‘भगवत् गीता’मा धिरज अनुबन्धित,1
1,पब्जी विश्वकपमा नेपाल दोस्रो,1
2,मोदीको मातृभक्ति : आमाको खुट्टा धोएर लिए आशीर्वाद,1
3,‘लोकतन्त्रप्रति प्रतिबद्ध नेतृत्व आउँछ’– निधि,1
4,'पथेर' र 'कदम'लाई पहिचान पुरस्कार,1
5,घाँस खेतीबाट वार्षिक २० लाख रुपियाँ गोकर्णकाे ...,1
6,वैदेशिक रोजगारीका लागि मलेसिया प्रमुख गन्तव्य,1
7,"युक्रेनीलाई बिजुली, पानीबाट वञ्चित गराउँदै रूस",1
8,सुनौलो सहकारीमा पुनः सापकोटा,1
9,अभ्यास खेलमा नेपाल पराजित,1


# suffle gareu data lai

In [14]:
suffle_data = cleaned_data.sample(frac=1)
suffle_data.head(10)

,Title,class
1244,तोरी खेतीमा किसानको आकर्षण बढेन,1
8378,"युरोप क्र्यास र बर्न्स, जब EU अधिकारीहरूले अन्...",0
1053,हेर्नुहोस्: फक्स न्यूजले गन कन्ट्रोलमा राष्ट्र...,0
1727,अर्थशास्त्रीहरूले ट्रम्पले विश्वव्यापी अर्थतन्...,0
3150,चीनमा पाँच हजार बढी सङ्क्रमित,1
6553,चीनले फेर्‍यो हङकङको चुनावी कानुन,1
687,ट्रम्प-टाउट विस्कन्सिन सम्झौताले करदाताहरूलाई ...,0
4457,अन्तमा: शेरिफ जो जातीय प्रोफाइलिङमा अवहेलनामा...,0
6902,नाइजेरियाको जेलबाट १८०० कैदी फरार,1
179,"मार्-ए-लागो नजिकै ""ब्ल्याक्स 4 ट्रम्प"" द्वारा ...",0


# Adding new index

In [16]:
# due to suffleing the index also gets suffeled.
# So reseting new index of the dataframe to its default integer_based indexing.
suffle_data.reset_index(inplace=True)
suffle_data.head()

,index,Title,class
0,1244,तोरी खेतीमा किसानको आकर्षण बढेन,1
1,8378,"युरोप क्र्यास र बर्न्स, जब EU अधिकारीहरूले अन्...",0
2,1053,हेर्नुहोस्: फक्स न्यूजले गन कन्ट्रोलमा राष्ट्र...,0
3,1727,अर्थशास्त्रीहरूले ट्रम्पले विश्वव्यापी अर्थतन्...,0
4,3150,चीनमा पाँच हजार बढी सङ्क्रमित,1


In [17]:
# deleting the old index column
suffle_data.drop(['index'],axis=1,inplace = True)
suffle_data.head(5)

,Title,class
0,तोरी खेतीमा किसानको आकर्षण बढेन,1
1,"युरोप क्र्यास र बर्न्स, जब EU अधिकारीहरूले अन्...",0
2,हेर्नुहोस्: फक्स न्यूजले गन कन्ट्रोलमा राष्ट्र...,0
3,अर्थशास्त्रीहरूले ट्रम्पले विश्वव्यापी अर्थतन्...,0
4,चीनमा पाँच हजार बढी सङ्क्रमित,1


# checking null value

In [19]:
suffle_data.isnull().sum()

Title    0
class    0
dtype: int64

# mixing both fake and real into 1

In [21]:
suffle_data.to_csv('mixed.csv', index=False)

In [22]:
data = pd.read_csv('mixed.csv')
data.head()

,Title,class
0,तोरी खेतीमा किसानको आकर्षण बढेन,1
1,"युरोप क्र्यास र बर्न्स, जब EU अधिकारीहरूले अन्...",0
2,हेर्नुहोस्: फक्स न्यूजले गन कन्ट्रोलमा राष्ट्र...,0
3,अर्थशास्त्रीहरूले ट्रम्पले विश्वव्यापी अर्थतन्...,0
4,चीनमा पाँच हजार बढी सङ्क्रमित,1


# Preprocessing

In [24]:
def wordopt(Title):
    
    
    # Title = Title.lower()
    Title = re.sub(r'[\[\]]', '', Title)  # Remove square brackets
    Title = re.sub(r'https?://\S+|www\.\S+', '', Title)  # Remove URLs
    Titel = re.sub(r'<.*?>', '', Title)  # Remove HTML tags
    Title = re.sub(f'[{re.escape(string.punctuation)}]', '', Title)  # Remove punctuation
    Title = re.sub(r'\n', '', Title)  # Remove newlines
    Title = re.sub(r'\w*\d\w*', '', Title)  # Remove words with digits
    Title = re.sub(r'\b[a-zA-Z]+\b', '', Title)  # This removes English words    
    return Title


In [25]:
data['Title2']= data['Title'].apply (wordopt)
data.head(50)

,Title,class,Title2
0,तोरी खेतीमा किसानको आकर्षण बढेन,1,तोरी खेतीमा किसानको आकर्षण बढेन
1,"युरोप क्र्यास र बर्न्स, जब EU अधिकारीहरूले अन्...",0,युरोप क्र्यास र बर्न्स जब अधिकारीहरूले अन्ततः...
2,हेर्नुहोस्: फक्स न्यूजले गन कन्ट्रोलमा राष्ट्र...,0,हेर्नुहोस् फक्स न्यूजले गन कन्ट्रोलमा राष्ट्रप...
3,अर्थशास्त्रीहरूले ट्रम्पले विश्वव्यापी अर्थतन्...,0,अर्थशास्त्रीहरूले ट्रम्पले विश्वव्यापी अर्थतन्...
4,चीनमा पाँच हजार बढी सङ्क्रमित,1,चीनमा पाँच हजार बढी सङ्क्रमित
5,चीनले फेर्‍यो हङकङको चुनावी कानुन,1,चीनले फेर्‍यो हङकङको चुनावी कानुन
6,ट्रम्प-टाउट विस्कन्सिन सम्झौताले करदाताहरूलाई ...,0,ट्रम्पटाउट विस्कन्सिन सम्झौताले करदाताहरूलाई ...
7,अन्तमा: शेरिफ जो जातीय प्रोफाइलिङमा अवहेलनामा...,0,अन्तमा शेरिफ जो जातीय प्रोफाइलिङमा अवहेलनामा ...
8,नाइजेरियाको जेलबाट १८०० कैदी फरार,1,नाइजेरियाको जेलबाट कैदी फरार
9,"मार्-ए-लागो नजिकै ""ब्ल्याक्स 4 ट्रम्प"" द्वारा ...",0,मार्एलागो नजिकै ब्ल्याक्स ट्रम्प द्वारा आयोजि...


In [26]:
data.columns

Index(['Title', 'class', 'Title2'], dtype='object')

# Tokenization

In [28]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
data['tokens1'] = data['Title2'].apply(word_tokenize)
data.head()

,Title,class,Title2,tokens1
0,तोरी खेतीमा किसानको आकर्षण बढेन,1,तोरी खेतीमा किसानको आकर्षण बढेन,"[तोरी, खेतीमा, किसानको, आकर्षण, बढेन]"
1,"युरोप क्र्यास र बर्न्स, जब EU अधिकारीहरूले अन्...",0,युरोप क्र्यास र बर्न्स जब अधिकारीहरूले अन्ततः...,"[युरोप, क्र्यास, र, बर्न्स, जब, अधिकारीहरूले, ..."
2,हेर्नुहोस्: फक्स न्यूजले गन कन्ट्रोलमा राष्ट्र...,0,हेर्नुहोस् फक्स न्यूजले गन कन्ट्रोलमा राष्ट्रप...,"[हेर्नुहोस्, फक्स, न्यूजले, गन, कन्ट्रोलमा, रा..."
3,अर्थशास्त्रीहरूले ट्रम्पले विश्वव्यापी अर्थतन्...,0,अर्थशास्त्रीहरूले ट्रम्पले विश्वव्यापी अर्थतन्...,"[अर्थशास्त्रीहरूले, ट्रम्पले, विश्वव्यापी, अर्..."
4,चीनमा पाँच हजार बढी सङ्क्रमित,1,चीनमा पाँच हजार बढी सङ्क्रमित,"[चीनमा, पाँच, हजार, बढी, सङ्क्रमित]"


# acessing the list of stopwords

In [30]:
stop = open('stopwords.txt', encoding='utf-8')  # Specify encoding
stopwords = []
for x in stop: # perform through each line of the txt file ma.
    x = x.replace('\n', '')  # Remove newline characters
    stopwords.append(x)      # Add to the stopwords list
stop.close()  # Close the file after reading
data.head(1)

,Title,class,Title2,tokens1
0,तोरी खेतीमा किसानको आकर्षण बढेन,1,तोरी खेतीमा किसानको आकर्षण बढेन,"[तोरी, खेतीमा, किसानको, आकर्षण, बढेन]"


In [31]:
stopwords

['',
 'आफ्न',
 'गर्नेछौ',
 'वाहे',
 'जसम',
 'सङ्गको',
 'चाहे',
 'जस्तै',
 'भनेर',
 'तिमीसँ',
 'चाहन्छौ',
 'जाहि',
 'सम्भ',
 'भन्छन्',
 'त्यस्तो',
 'प्रत्येक',
 'कत',
 'थरि',
 'त्यस्तै',
 'कति',
 'गर्छ',
 'आफैल',
 'सायद',
 'त्यसै',
 'सुरुमै',
 'भीत्र',
 'पटक',
 'अलग',
 'अक्स',
 'गर्दछ',
 'जसको',
 'नत्र',
 'राख्',
 'थिएन',
 'छु',
 'यद',
 'हो',
 'अर्थात',
 'गर्द',
 'छौं',
 'पाँ',
 'होकि',
 'भन्न',
 'दि',
 'नै',
 'पछिल्ल',
 'सध',
 'आत्',
 'पक्क',
 'तेस्कार',
 'पर्नेमा',
 'कम से क',
 'एउटा',
 'सक्नुहुन्छ',
 'भन्द',
 'उनीहरु',
 'म',
 'उसलाई',
 'रे',
 'सम',
 'बिच',
 'त्यत्तिकै',
 'जस्त',
 'पनि',
 'यसैल',
 'वाहेक',
 'ठाउँमा',
 'प्रत्ये',
 'गय',
 'अल',
 'निर्दिष्',
 'कोही',
 'यस',
 'आफैलाई',
 'द्वार',
 'यस्तै',
 'अर्क',
 'मैल',
 'एकद',
 'छिन्',
 'कस्त',
 'देखेको',
 'समयम',
 'तिनै',
 'चाले',
 'पर्याप्त',
 'सम्म',
 'जस',
 'स्पष्',
 'यस्त',
 'अरू',
 'दश',
 'समेत',
 'सोध्',
 'त्य',
 'ऊ',
 'तसरी',
 'मात्र',
 'जबकि',
 'तिनीहरूले',
 'किनभने',
 'उदाहरण',
 'नभएको',
 'केही',
 'साय',
 'तिनीहरूलाई',
 'बाहि

# Removing stopwords

In [33]:
def remove_stopwords(x):
    return ''.join([word for word in str(x).split() if word not in stopwords])
data['title3']= data['tokens1']. apply(lambda x :remove_stopwords(x))# for each row in the tokens column 
#lambda function is called with x being the value of that row.
#The result of the lambda function (which is the value returned by remove_stopwords(x)) is assigned to the new column title3.
data.head(5)

,Title,class,Title2,tokens1,title3
0,तोरी खेतीमा किसानको आकर्षण बढेन,1,तोरी खेतीमा किसानको आकर्षण बढेन,"[तोरी, खेतीमा, किसानको, आकर्षण, बढेन]","['तोरी','खेतीमा','किसानको','आकर्षण','बढेन']"
1,"युरोप क्र्यास र बर्न्स, जब EU अधिकारीहरूले अन्...",0,युरोप क्र्यास र बर्न्स जब अधिकारीहरूले अन्ततः...,"[युरोप, क्र्यास, र, बर्न्स, जब, अधिकारीहरूले, ...","['युरोप','क्र्यास','र','बर्न्स','जब','अधिकारीह..."
2,हेर्नुहोस्: फक्स न्यूजले गन कन्ट्रोलमा राष्ट्र...,0,हेर्नुहोस् फक्स न्यूजले गन कन्ट्रोलमा राष्ट्रप...,"[हेर्नुहोस्, फक्स, न्यूजले, गन, कन्ट्रोलमा, रा...","['हेर्नुहोस्','फक्स','न्यूजले','गन','कन्ट्रोलम..."
3,अर्थशास्त्रीहरूले ट्रम्पले विश्वव्यापी अर्थतन्...,0,अर्थशास्त्रीहरूले ट्रम्पले विश्वव्यापी अर्थतन्...,"[अर्थशास्त्रीहरूले, ट्रम्पले, विश्वव्यापी, अर्...","['अर्थशास्त्रीहरूले','ट्रम्पले','विश्वव्यापी',..."
4,चीनमा पाँच हजार बढी सङ्क्रमित,1,चीनमा पाँच हजार बढी सङ्क्रमित,"[चीनमा, पाँच, हजार, बढी, सङ्क्रमित]","['चीनमा','पाँच','हजार','बढी','सङ्क्रमित']"


# Stemming the language

In [35]:
# import snowballstemmer
# test_words = ["नेपालीको", "आफ्नै", "स्तेम्मेर", "यसलाई", "अनुमोदनको"]
#  stemmed_words = nepali_stemmer.stemWords(test_words)
#  print("Original Words:", test_words)
#  print("Stemmed Words:", stemmed_words)


In [36]:
# import re
# import snowballstemmer

# # Initialize the NepaliStemmer instance
# nepali_stemmer = snowballstemmer.NepaliStemmer()

# def preprocess_text(text):
#     if isinstance(text, str):
#         # Remove punctuation and extra spaces
#         text = re.sub(r'[^\w\s]', '', text)
#         return text.strip()
#     return text

# def stem_text(text):
#     text = preprocess_text(text)
#     if isinstance(text, str):
#         return " ".join(nepali_stemmer.stemWords(text.split()))
#     return text

# # Apply the preprocessing and stemming to the 'title3' column
# data['data4'] = data['title3'].apply(stem_text)

# # Display results
# print(data[['title3', 'data4']].head(20))
# data.head(20)

In [37]:
data.columns

Index(['Title', 'class', 'Title2', 'tokens1', 'title3'], dtype='object')

In [38]:
x = data['title3']
y = data['class']

In [39]:
x

0              ['तोरी','खेतीमा','किसानको','आकर्षण','बढेन']
1        ['युरोप','क्र्यास','र','बर्न्स','जब','अधिकारीह...
2        ['हेर्नुहोस्','फक्स','न्यूजले','गन','कन्ट्रोलम...
3        ['अर्थशास्त्रीहरूले','ट्रम्पले','विश्वव्यापी',...
4                ['चीनमा','पाँच','हजार','बढी','सङ्क्रमित']
                               ...                        
19509              ['जनामा','सङ्क्रमण','सङ्क्रमण','मुक्त']
19510    ['गल्फ','कोर्सबाट','बाहिर','ट्रम्पले','एफबीआई'...
19511    ['भिडियो','कार्लीले','उनीसँग','डिमेन्टेड','अनु...
19512    ['प्रधानन्यायाधीशको','बहिर्गमनबारे','छलफल','गर...
19513          ['नेपालमा','पानासोनिकको','सोरुम','स्थापना']
Name: title3, Length: 19514, dtype: object

In [40]:
y

0        1
1        0
2        0
3        0
4        1
        ..
19509    1
19510    0
19511    0
19512    1
19513    1
Name: class, Length: 19514, dtype: int64

In [41]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.10)

In [42]:
x_train.shape

(17562,)

In [43]:
x_test.shape

(1952,)

In [44]:
x_train.head(50)

18031    ['भाग्य','कमाउन','शरिया','कानून','प्रयोग','गर्...
16886                  ['बाख्रापालनबाट','मनग्य','आम्दानी']
10124    ['बहसमा','ताम्बा','दर्शन','मातृभाषा','र','लोकस...
967      ['नक्कली','भारतीय','एलिजाबेथ','वारेनले','डिएनए...
2848     ['वाग्मतीमा','माओवादी','केन्द्रले','सरकारलाई',...
5047     ['क्लिन्टन','कभरअप','किन','मेजर','क्लिन्टन','क...
4088     ['ज्याला','नपाउँदा','मजदुरले','मनाउन','पाएनन्'...
3905     ['सेबोनको','अध्यक्ष','सिफारिस','समिति','पुनः',...
3714     ['सिद्धार्थ','बैंकको','औं','वार्षिकोत्सव','सम्...
7508     ['वाह','मिलवाउकी','स्कूल','अफ','इन्जिनियरिङ्का...
19077    ['भिडियो','अवश्य','हेर्नुहोस','ओबामा','र','उनक...
464      ['संयुक्त','राष्ट्र','समाचार','एजेन्सीले','ट्र...
7436     ['ट्रम्प','अब','क्रिस','क्रिस्टीको','शुगर','ड्...
3050     ['‘','राष्ट्रपतिलाई','विधेयक','रोक्ने','र','वि...
1531     ['भारतको','वेस्ट','इन्डिजविरुद्ध','लगातार','छै...
18738    ['फोहोर','पोखरी','र','ले','भर्खरै','पुष्टि','ग...
8533     ['जबजलाई','भजनकीर्तनको','रूपमा','प्रयोग','गर्न.

In [45]:
y_test.head(50)

16934    1
12524    0
1333     0
502      1
15351    1
6091     1
12349    0
15687    1
14742    1
1544     1
2965     1
11170    0
7976     1
1421     0
5362     1
7254     0
2449     0
13768    1
1085     0
16791    0
15691    1
10905    0
14327    1
16811    0
15946    1
10700    0
17083    0
3758     1
11273    1
9441     1
8728     0
10493    1
16957    1
7247     0
15355    0
12651    1
14673    0
3018     0
8826     0
17703    1
7521     1
18184    1
11079    1
9364     0
9519     0
11685    1
13440    0
16392    1
5072     0
11426    1
Name: class, dtype: int64

In [46]:
# here i built different file which contain only testing data
x_test.to_csv('testing_data.csv', index=False)

In [47]:
# similarly building file for training data
x_train.to_csv('training_data.csv', index=False)

# Vectorizing the text data into numerical


In [49]:
# #  here due to the nepali datasets the system doest tfidf the data.
# # so for that i have to convert the data into numerical featues using countvectorizer
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer(ngram_range=(1,2)).fit(x_train)
# x_train_vectorized = vectorizer.transform(x_train)
# print(x_train_vectorized)
# x_train_vectorized.shape

In [50]:
# vectorizer = CountVectorizer(ngram_range=(1,2))
# x_train_vectorized = vectorizer.fit_transform(x_train)

# Transforming the text data

In [52]:
#converting the text into vector The TfidfVectorizer is used to convert a collection of text documents into numerical vectors,
#that can be fed into machine learning models. It does this by calculating the TF-IDF for each word in the documents.
vectorization = TfidfVectorizer() 
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [53]:
xv_train

<17562x4028 sparse matrix of type '<class 'numpy.float64'>'
	with 137099 stored elements in Compressed Sparse Row format>

# Model Training

In [55]:
model = LogisticRegression()
model.fit(xv_train,y_train)

LogisticRegression()

In [56]:
pred_lr=model.predict(xv_test)

In [57]:
model.score(xv_test, y_test)

0.920594262295082

In [58]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92       990
           1       0.91      0.93      0.92       962

    accuracy                           0.92      1952
   macro avg       0.92      0.92      0.92      1952
weighted avg       0.92      0.92      0.92      1952



In [59]:
def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Real News"

def manual_testing(news):
    # Create a DataFrame from the input news
    testing_news = {"title3": [news]}
    new_def_test = pd.DataFrame(testing_news)

    # Apply preprocessing using wordopt (assuming it's a defined function)
    new_def_test["title3"] = new_def_test["title3"].apply(wordopt) 

    # Transform the text using vectorization
    new_x_test = new_def_test["title3"]
    new_xv_test = vectorization.transform(new_x_test)

    # Perform prediction using Logistic Regression
    pred_model = model.predict(new_xv_test)

    # Output the prediction result
    return print("\n\nLogistic Regression Prediction: {}".format(output_label(pred_model[0])))


In [60]:
news_input = "['उफ्','मिडियाले','झूट','बोल्योसैन्य','सदस्यहर..."
manual_testing(news_input)



Logistic Regression Prediction: Fake News


# Confusion metrics

In [62]:
from sklearn.metrics import confusion_matrix

# confusion metrics
confusion_matrix(y_test, pred_lr,labels=[0, 1])

array([[902,  88],
       [ 67, 895]], dtype=int64)

# saving the model using pickle

In [64]:
import pickle


In [65]:
# create an iterator object with write permission - model.pkl
with open('model_pkl', 'wb') as files:
    pickle.dump(model, files)

In [66]:
# load saved model
with open('model_pkl' , 'rb') as file:
    model= pickle.load(file)

# vectorized form ma lageko 

In [68]:
input_text = ["उफ्','मिडियाले','झूट','बोल्योसैन्य','सदस्यहर.."]
input_vectorized = vectorization.transform(input_text)

In [69]:
# model.predict([["'राष्ट्रपति','पदका','उम्मेद्वार','मार्को','रु.."]])
prediction = model.predict(input_vectorized)
print(prediction)

[0]
